In [1]:
from test import load_data_sparse
from SGD import *
from ALS import *
from surprise_models import *
from surprise.model_selection import train_test_split
import pickle
from simple_models import *
from helper import calculate_rmse

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"
test_name = "data/sampleSubmission.csv"

Load trainset:

In [4]:
sp_ratings, pd_ratings = load_data_sparse(data_name, False)

Convert it into surprise

In [5]:
spr_data = pandas_to_surprise(pd_ratings)

Split into test and dataset

In [6]:
trainset, testset = train_test_split(spr_data, test_size=.1, random_state=988)

In [7]:
pickle.dump(trainset, open('trainset.pkl', 'wb'))
pickle.dump(testset, open('testset.pkl', 'wb'))

get labels

In [8]:
_,_,labels = get_testset_indices(testset)

In [9]:
pickle.dump(labels, open('labels.pkl', 'wb'))

In [10]:
%%time
baseline = surprise_baseline(trainset, testset)

Estimating biases using als...
Wall time: 5.31 s


In [11]:
pickle.dump(baseline, open(PATH+'baseline.pkl', 'wb'))

In [12]:
calculate_rmse(labels, baseline)

0.9961241888955135

In [13]:
%%time
svd = surprise_SVD(trainset, testset)

Wall time: 42 s


In [14]:
pickle.dump(svd, open('svd.pkl', 'wb'))

In [15]:
calculate_rmse(labels, svd)

0.997766335033669

In [16]:
%%time
so = surprise_slopeOne(trainset, testset)

Wall time: 26.4 s


In [17]:
pickle.dump(so, open('so.pkl', 'wb'))

In [18]:
calculate_rmse(labels, so)

0.9969080076726844

In [19]:
%%time
bsknn = surprise_baselineKNN(trainset, testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Wall time: 1min 1s


In [20]:
pickle.dump(bsknn, open('bsknn.pkl', 'wb'))

In [21]:
calculate_rmse(labels, bsknn)

0.9856890199729544

In [22]:
num_features = 40  # K in the lecture notes
lambda_user = 0.1
lambda_film = 0.1
stop_criterion = 1e-4

In [23]:
%%time
als = ALS_CV(trainset, testset, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using ALS...
RMSE: 1.0884265270224045.
RMSE: 1.0448632788224261.
RMSE: 1.0075310351133466.
RMSE: 0.977725106882568.
RMSE: 0.9578063012519292.
RMSE: 0.943936611734106.
RMSE: 0.934018006599574.
RMSE: 0.9268514492599935.
RMSE: 0.9215935145245444.
RMSE: 0.9176478820904929.
RMSE: 0.9146178986800569.
RMSE: 0.9122430834881904.
RMSE: 0.9103489980977701.
RMSE: 0.9088156355449482.
RMSE: 0.907558281206095.
RMSE: 0.9065157002156952.
RMSE: 0.905642666984162.
RMSE: 0.9049051437566374.
RMSE: 0.9042771024724575.
RMSE: 0.9037383770215942.
RMSE: 0.9032731715348948.
RMSE: 0.9028689964758839.
RMSE: 0.902515892332675.
RMSE: 0.9022058529097867.
RMSE: 0.9019323911571836.
RMSE: 0.9016902091543004.
RMSE: 0.9014749455324177.
RMSE: 0.9012829811943149.
RMSE: 0.9011112892983215.
RMSE: 0.9009573190310811.
RMSE: 0.9008189052404403.
RMSE: 0.900694197874881.
RMSE: 0.9005816065853541.
RMSE: 0.9004797569276486.
RMSE: 0.9003874554430887.
Iteration stopped, as iteration criterion 0.0001 was 

In [24]:
pickle.dump(als, open('als.pkl', 'wb'))

In [25]:
calculate_rmse(labels, als)

0.9842761153117054

In [26]:
%%time
sgd = matrix_factorization_SGD_CV(trainset, testset, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using SGD...
RMSE: 1.0428992175794367.
RMSE: 1.017043995115573.
RMSE: 1.010158056295032.
RMSE: 1.0060903480513685.
RMSE: 1.0047221572192553.
RMSE: 1.0039966006712449.
RMSE: 1.003678416198583.
RMSE: 1.0035711474648574.
RMSE: 1.0035330294859752.
Wall time: 6min 33s


In [27]:
pickle.dump(sgd, open('sgd.pkl', 'wb'))

In [28]:
calculate_rmse(labels, sgd)

1.0228761455871391

In [29]:
%%time
svdpp = surprise_SVDpp(trainset, testset)

Wall time: 50min 55s


In [30]:
pickle.dump(svdpp, open('svdpp.pkl', 'wb'))

In [31]:
calculate_rmse(labels, svdpp)

0.9974433490996525

In [32]:
%%time
globalmean = global_mean(trainset, testset)

Wall time: 8.39 s


In [33]:
pickle.dump(globalmean, open('globalmean.pkl', 'wb'))

In [34]:
calculate_rmse(labels, globalmean)

1.117181294724137

In [35]:
%%time
usermean = user_mean(trainset, testset)

Wall time: 54 s


In [36]:
pickle.dump(usermean, open('usermean.pkl', 'wb'))

In [37]:
calculate_rmse(labels, usermean)

1.0277625350651283

In [38]:
%%time
itemmean = item_mean(trainset, testset)

Wall time: 13.9 s


In [39]:
pickle.dump(itemmean, open('itemmean.pkl', 'wb'))

In [40]:
calculate_rmse(labels, itemmean)

1.0933093510076437

In [41]:
%%time 
basicknn = surprise_basicKNN(trainset, testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 4min 37s


In [42]:
pickle.dump(basicknn, open('basicknn.pkl', 'wb'))

In [43]:
calculate_rmse(labels, basicknn)

1.0218854894245444

In [ ]:
globalmedian = global_median(trainset, testset)
usermedian = user_median(trainset, testset)
itemmedian = item_median(trainset, testset)

In [ ]:
pickle.dump(globalmedian, open('globalmedian.pkl', 'wb'))
pickle.dump(usermedian, open('usermedian.pkl', 'wb'))
pickle.dump(itemmedian, open('itemmedian.pkl', 'wb'))

In [ ]:
calculate_rmse(labels, globalmedian)
calculate_rmse(labels, usermedian)
calculate_rmse(labels, itemmedian)